In [26]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Function to generate data
def generate_data():
    x = torch.arange(-2, 2, 0.2).view(-1, 1)
    func = 5 * x
    noise = 1.7 * torch.randn(x.size())
    y = func + noise
    return x, func, y

# Forward function
def forward(x, w):
    return w * x

# Regularized loss function (L2 regularization)
def regularized_loss(y_pred, y, w, lambda_val):
    mse_loss = torch.mean((y_pred - y) ** 2)
    l2_penalty = lambda_val * torch.sum(w**2)
    return mse_loss + l2_penalty

# Training function
def train(x, y, w, step_size, iterations, epsilon, lambda_val):
    loss_list = []
    for i in range(iterations):
        # Forward pass
        y_pred = forward(x, w)
        
        # Regularized loss
        loss = regularized_loss(y_pred, y, w, lambda_val)
        loss_list.append(loss.item())
        
        # Backward pass
        loss.backward()
        
        # Check gradient for early stopping
        if torch.abs(w.grad).item() < epsilon:
            print(f"Stopping early at iteration {i + 1}: gradient is near zero")
            break
        
        # Update parameters
        with torch.no_grad():
            w -= step_size * w.grad
        
        # Zero out gradients
        w.grad.zero_()
    
    return w, loss_list

# Grid Search over different lambda values
def grid_search_lambda(x, y):
    lambda_values = [0.001, 0.01, 0.1, 1, 10]
    best_lambda = None
    best_loss = float('inf')
    
    for lambda_val in lambda_values:
        # Initialize weight parameter
        w = torch.tensor(-10.0, requires_grad=True)
        
        # Train model with current lambda
        w, loss_list = train(x, y, w, step_size=0.1, iterations=3000, epsilon=1e-6, lambda_val=lambda_val)
        
        # Compute final loss and track the best lambda
        final_loss = loss_list[-1]
        print(f"Lambda: {lambda_val}, Final Loss: {final_loss}")
        
        if final_loss < best_loss:
            best_loss = final_loss
            best_lambda = lambda_val
    
    print(f"Best Lambda: {best_lambda}")
    return best_lambda

# Main function to run grid search
def main():
    # Step 1: Generate data
    x, func, y = generate_data()
    
    # Step 2: Perform Grid Search for optimal lambda
    best_lambda = grid_search_lambda(x, y)
    
    # Step 3: Visualize the results
    print(f"Optimal lambda: {best_lambda}")

main()


Stopping early at iteration 55: gradient is near zero
Lambda: 0.001, Final Loss: 2.5483407974243164
Lambda: 0.01, Final Loss: 2.7949376106262207
Lambda: 0.1, Final Loss: 5.091371536254883
Stopping early at iteration 30: gradient is near zero
Lambda: 1, Final Loss: 18.34002685546875
Lambda: 10, Final Loss: nan
Best Lambda: 0.001
Optimal lambda: 0.001
